In [1]:
filenames = []

In [2]:
import nltk
import pandas as pd
import numpy as np
import re
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")
K = 30


In [3]:
#import the query relevance results and store it in a pandas datafrom
filename = 'cacm/query.csv'
file = open(filename,'r')
text = re.sub('[0-9]+', '', file.read())
file.close()
queries = text.split('\n')

#queries = pd.read_csv(filename)
#queries.columns = ['No.','Text']

In [4]:
# import the common words to be used as stop words
filename = 'cacm/common_words'
file = open(filename,'r')
text = file.read()
file.close()
common_words = text.split('\n')

In [5]:

def createFileForArticle(article,count):
    file_name = 'cacm/cacm_seperated/cacm_' + str(count)
    lines = article.split('\n')
    new_file = open(file_name,'w+')
    filenames.append(file_name)
    for line in lines:     
       
        if(line.startswith('.X')):
            break
        if(not(line.startswith(('.T','.B','.A','.N','.X','.I')))):   
            new_file.write(line)
        

In [6]:
filename_cacm = 'cacm/cacm.all'
file = open(filename_cacm,'r')
text = file.read()
file.close()
count = 0
articles = text.split('.I ')
for article in articles:
    count+=1
    createFileForArticle(article,count)
    

In [7]:
# retrieve all files in cacm_seperated folder
import re
corpus=[]

for entry in filenames:
    file = open(str(entry),'rt')
    text = re.sub('[0-9]+', '', file.read())
    corpus.append(text)
    file.close()


In [8]:
stopWords       = open ('cacm/common_words', 'rt').read()
stopWordList    = stopWords.split('\n')


In [9]:
def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems


In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

vectorizer = TfidfVectorizer(stop_words=tokenize_and_stem(str(common_words)),tokenizer=tokenize_and_stem)
docs_tfidf = vectorizer.fit_transform(corpus)
filename = 'cacm/results/result.txt'
file = open(filename,'w+')
def get_tf_idf_query_similarity(vectorizer, docs_tfidf, query,query_no):
    query_tfidf = vectorizer.transform([query])
    cosineSimilarities = cosine_similarity(query_tfidf, docs_tfidf).flatten()
    sorted_similarity = sorted(((v, i+1) for i, v in enumerate(cosineSimilarities)), reverse=True)
    for t in sorted_similarity[:K]:
        file.write(str(query_no) + " " +str(t[1]) + "\n")
    return cosineSimilarities

C:\Users\ZETTA\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:383: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['anywh', 'el', 'elsewh', 'everywh', 'ind', 'otherwi', 'plea', 'somewh'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


In [19]:
print(vectorizer.get_feature_names())

["'+p", '+-f', '+b', '+k', '+logn', '+o', '+q', '+sin', '+size', '+u', '+x', '+y', ',and/or', '-.wbase', '-.win', '-.wthe', '-a', '-an', '-bit', '-bitcel', '-bitcod', '-byte', '-cacm', '-charact', '-column', '-construct', '-d', '-d.wan', '-digit', '-digithexadecim', '-dimension', '-fold', '-gate', '-georg', '-ii', '-iicacm', '-iiicacm', '-impressionsof', '-j', '-line', '-ln', '-locat', '-multiprogram', '-node', '-p', '-part', '-pdp-system', '-plane', '-point', '-space', '-stabil', '-stability.cacm', '-stabl', '-t', '-track', '-tree', '-treeswith', '-u', '-with', '-word', '-wordvector', '-year', '.c', '.c.ca', '.ca', '.cacm', '.cajb', '.kalphamer', '.kcalendar', '.kcomput', '.kcoulomb', '.kdata', '.khidden-lin', '.kinput-output', '.kinterchang', '.klinear', '.kreal', '.kstandard', '.ktabulating-card', '.kthis', '.kusa', '.use', '.w', '.wa', '.walgol', '.wan', '.wbefor', '.wbi', '.wcertain', '.wcomput', '.wcontext', '.wdescript', '.wdiffer', '.weach', '.weffici', '.weven', '.wfollow', '.

In [11]:
results = []

for index,query in enumerate(queries):
    similarity = get_tf_idf_query_similarity(vectorizer,docs_tfidf,query,index+1)
    sorted_similarity = sorted(((v, i+1) for i, v in enumerate(similarity)), reverse=True)
    #print(index+1,(sorted(similarity,reverse=True)[:3])
    #print(index+1,similarity)
    results.append(index+1)
    results.append(sorted_similarity[:1])
    #results.append((-similarity).argsort()[:5]+1)
    #results.append(sorted(similarity,reverse=True)[:3])

file.close()

In [12]:
#https://stackabuse.com/python-for-nlp-creating-a-rule-based-chatbot/
#https://stackoverflow.com/questions/57124182/how-to-fix-stopwords-preprocessing-inconsistency

Now it is time to measure accuracy and precision

In [13]:
#import the query relevance results and store it in a pandas datafrom
filename = 'cacm/qrels.text'
qrels = pd.read_csv(filename,sep="\s+")
qrels.columns = ['query', 'document','score','relevance']
qrels.drop(columns=['score','relevance'],axis=1,inplace=True)

In [14]:
#import the query relevance results and store it in a pandas datafrom
filename = 'cacm/results/result.txt'
preds = pd.read_csv(filename,sep="\s+")
preds.columns = ['query', 'document']


In [15]:
def get_precision(query_id):
    no_of_relevant_documents = qrels['query'].loc[qrels['query'] == query_id].count() or 1
    no_of_documents_retrieved= K
    X = (preds['document'].loc[preds['query'] == query_id])
    y = (qrels['document'].loc[qrels['query'] == query_id])
    X_y_merged = pd.merge(X, y, how ='inner', on =['document'])
    no_of_relevant_documents_retrieved = X_y_merged['document'].count()
    #no_of_relevant_documents_retrieved = (preds.loc[preds['query'] == query_id])
    
    precision = no_of_relevant_documents_retrieved / K
    recall = no_of_relevant_documents_retrieved / no_of_relevant_documents
    return precision,recall


In [16]:
sum_precision = 0
sum_recall = 0
for query_id in range(1,len(queries)):
    p,r = get_precision(query_id)
    sum_precision+=p
    sum_recall+=r
print(sum_precision/64,sum_recall/64)

0.15572916666666667 0.3788354540708639
